# Minimal OpenRouter LLM Demo

This notebook shows **the absolute basics** of calling an LLM via the [OpenRouter](https://openrouter.ai) API using Python.

**What you'll need**
1. A file named `.env` in the same folder as this notebook with a line like:
   ```bash
   OPENROUTER_API_KEY=sk-or-v1_...
   ```
2. An internet connection (calls go to `https://openrouter.ai/api/v1`).

We'll keep things minimal: install deps, load the API key from `.env`, make a single non-streaming request, then a streaming request. We also show how to change models and parameters.


In [ ]:
# (1) Install minimal dependencies (uncomment to run in a fresh environment)
%pip install requests python-dotenv --quiet

In [ ]:
# (2) Load your API key from .env and set up headers
import os, requests
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv("OPENROUTER_API_KEY")
assert API_KEY, "Missing OPENROUTER_API_KEY in your .env file"

BASE_URL = "https://openrouter.ai/api/v1/chat/completions"

# Optional headers help attribute your app in OpenRouter's dashboards
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
    # Optional but recommended (replace with your course URL or localhost)
    "HTTP-Referer": "http://localhost",  # shown in OpenRouter rankings
    "X-Title": "ML Class LLM Demo",      # a friendly app title
}

## Non‑streaming: single request → single JSON response
This mirrors the standard OpenAI-style Chat Completions interface. Change the `model` to any model you have access to in OpenRouter (e.g., `openai/gpt-4o-mini`, `anthropic/claude-3.5-sonnet`, `google/gemini-1.5-pro`).

In [ ]:
payload = {
    "model": "openai/gpt-4o-mini",  # pick any model from https://openrouter.ai/models
    "messages": [
        {"role": "system", "content": "You are a concise teaching assistant."},
        {"role": "user", "content": "Explain gradient descent in one sentence."},
    ],
    # Optional knobs (common across providers via OpenRouter):
    "temperature": 0.7,
    "max_tokens": 256,
}

resp = requests.post(BASE_URL, headers=HEADERS, json=payload, timeout=60)
resp.raise_for_status()
data = resp.json()
text = data["choices"][0]["message"]["content"]
print(text)

# (Optional) usage metadata
data.get("usage", {})

## Minimal helper function
A tiny helper to keep classroom examples tidy.

In [ ]:
def chat(prompt, model="openai/gpt-4o-mini", system="You are helpful and concise.", **params):
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ],
    }
    payload.update(params)
    r = requests.post(BASE_URL, headers=HEADERS, json=payload, timeout=60)
    r.raise_for_status()
    out = r.json()
    return out["choices"][0]["message"]["content"].strip()

print(chat("Name three common activation functions."))

## Streaming (SSE): print tokens as they arrive
Set `stream=True` to receive an SSE stream and print chunks as they come in. Ignore comment heartbeats that start with `:`.

In [ ]:
import json, sys

stream_payload = {
    "model": "openai/gpt-4o-mini",
    "messages": [
        {"role": "system", "content": "You are a concise teaching assistant."},
        {"role": "user", "content": "In two bullet points, what is overfitting?"},
    ],
    "stream": True,
    "temperature": 0.2,
}

with requests.post(BASE_URL, headers=HEADERS, json=stream_payload, stream=True, timeout=300) as r:
    r.raise_for_status()
    for line in r.iter_lines():
        if not line:
            continue
        # SSE comment/heartbeat lines start with ':' — ignore them
        if line.startswith(b":"):
            continue
        if line.startswith(b"data: "):
            chunk = line[len(b"data: "):].decode("utf-8")
            if chunk.strip() == "[DONE]":
                break
            event = json.loads(chunk)
            delta = event["choices"][0].get("delta", {})
            content = delta.get("content")
            if content:
                print(content, end="", flush=True)
print()

## Switching models & parameters
Swap the `model` string for any model you have access to (see [openrouter.ai/models](https://openrouter.ai/models)). Common knobs like `temperature`, `max_tokens`, `top_p`, and `frequency_penalty` are supported.

> Tip: If a model doesn't support a certain parameter, OpenRouter simply ignores it rather than failing.

In [ ]:
print(chat(
    "Write a 1‑sentence analogy for convolution in CNNs.",
    model="anthropic/claude-3.5-sonnet",  # try another provider/model
    temperature=0.5,
))

---
**That’s it — minimal and classroom‑friendly.**

**Safety note**: Don’t commit your `.env` file to version control. In a shared environment, consider using per‑student keys or a small proxy service so keys aren’t exposed on client machines.
